In [1]:
VERSION

v"1.10.5"

In [2]:
] activate .

  Activating new project at `~/TRIXI`


In [3]:
using OrdinaryDiffEq
using Trixi
using Plots

# Nonlinear Kerr Effect

This notebook describes how to use `Trixi.jl` to simulate the Slowly-Varying-Envelope-Approximation (SVEA) and Paraxial Wave Equations for a guassian beam including optical absorption and the nonlinear Kerr effect.

This notebook is based off the previous notebook for the SVEA Paraxial wave equation and many things will be reused. 

# Adding Longitudinal Dispersion and Nonlinear Instantaneous Refractive Index (Kerr Effect)

In the previous notebook, we used the textbook *Fundamentals of Nonlinear Optics* by Powers. We start from the same equation. The numbers of these equations will continue from the previous notebook.

The full time dependent paraxial wave equation is given by Equation 10.79 in Powers: 

\begin{equation}
    \frac{\partial}{\partial t} A + \frac{i v_{g}}{2 k_{0} v_{g}^{2}} \left. \left( 1 - k_{0} \frac{\partial v_{g}}{\partial \omega}  \right)\right|_{\omega_{0}} \frac{\partial^2}{\partial t^2} A  + v_{g} \frac{\partial}{\partial z} A - i\frac{v_{g}}{2 k_{0}} \nabla^{2} A = \frac{i \mu_{0} \omega_{0}^{2}}{2 k_{0}} P_{NL} e^{- i k_{0} z} 
    \tag{20}
\end{equation}

In Boyd's textbook, Equation 7.5.23, the group velocity dispersion (which is usually labeled as either $\kappa_2$ or $\beta_{2}$ ) is given by: 

\begin{equation}
    \begin{aligned}
    \kappa_{2} &= \left. \left( \frac{d^{2} k}{d \omega^{2}} \right) \right|_{\omega = \omega_{0}} \\
    &= \frac{d}{d \omega} \left. \left( \frac{1}{v_{g}\left(  \omega \right)}   \right) \right|_{\omega = \omega_{0}} \\
    &= \left. \left( \frac{1}{v_{g}^{2}\left(  \omega \right)}  \frac{d v_{g}\left(  \omega \right)}{d \omega} \right)  \right|_{\omega = \omega_{0}}
    \end{aligned}
\end{equation}

We can add optical absorption and Kerr effect:

\begin{equation}
    \frac{\partial}{\partial t} A + \frac{i}{2}\left( \frac{1}{k_{0} v_{g}} - v_{g} \kappa_{2}  \right)  \frac{\partial^2}{\partial t^2} A + \frac{\alpha}{2} A  + v_{g} \frac{\partial}{\partial z} A - i\frac{v_{g}}{2 k_{0}} \nabla^{2} A = i k_{0} n_{2} I A
    \tag{21}
\end{equation}

But we'll make the substitution for the full velocity dispersion with $\beta_{2}$:

\begin{equation}
    \frac{\partial}{\partial t} A + \frac{i}{2}\beta_{2}  \frac{\partial^2}{\partial t^2} A  + v_{g} \frac{\partial}{\partial z} A - i\frac{v_{g}}{2 k_{0}} \nabla^{2} A = -\frac{\alpha}{2} A + i k_{0} n_{2} I A
    \tag{22}
\end{equation}

where $n_{2}$ is the instantaneous second order nonlinear refractive index. More can be found in Boyd and this wikipedia article: https://en.wikipedia.org/wiki/Kerr_effect

We put the optical absorption (extinction coefficient term) and the nonlinear Kerr effect on the RHS because we will use `Trixi.jl` function for source terms.

The intensity $I$ is:

\begin{equation}
    \begin{aligned}
    I &= \frac{v_{p} \epsilon_{0} \epsilon_{r} \mu_{r} }{2} \left| E \right|^{2} = \frac{c \epsilon_{0} n}{2}\left| E \right|^{2} \\
    &= \frac{c \epsilon_{0}}{2} \sqrt{\frac{\epsilon_{r}}{\mu_{r}}} \;\;\left| E \right|^{2}
    \end{aligned}
\end{equation}

in SI units. In CGS units the formula is: 

\begin{aligned}
    I = \frac{c}{8 \pi} \sqrt{\frac{\epsilon_{r}}{\mu_{r}}}  \;\; \left| E \right|^{2}
\end{aligned}

We need to set this up into single derivatives in time so we have to re-write this with the following substitutions:

\begin{equation}
    \begin{aligned}
    B = \partial_{t} A
    \end{aligned}
\end{equation}

Then our equation becomes a coupled system: 

\begin{equation}
    \begin{aligned}
    \frac{i}{2}\beta_{2}  \frac{\partial}{\partial t} B  + v_{g} \frac{\partial}{\partial z} A - i\frac{v_{g}}{2 k_{0}} \nabla^{2} A &= - B -\frac{\alpha}{2} A + i k_{0} n_{2} I A \\
    \partial_{t} A &= B
    \end{aligned}
    \tag{23}
\end{equation}

Again, this needs to be split into the real and imaginary parts: 

\begin{equation}
    \begin{aligned}
    A(\mathbf{x},t) &= A_{r}(\mathbf{x},t) + i A_{i}(\mathbf{x},t) \\
    B(\mathbf{x},t) &= B_{r}(\mathbf{x},t) + i B_{i}(\mathbf{x},t)
    \end{aligned}
\end{equation}

\begin{equation}
    \begin{aligned}
    \frac{i}{2}\beta_{2}  \frac{\partial}{\partial t} \left( B_{r} + i B_{i} \right) + v_{g} \frac{\partial}{\partial z} \left( A_{r} + i A_{i} \right) - i\frac{v_{g}}{2 k_{0}} \nabla^{2} \left( A_{r} + i A_{i} \right) &= - \left( B_{r} + i B_{i} \right) -\frac{\alpha}{2} \left( A_{r} + i A_{i} \right) + i k_{0} n_{2} I \left( A_{r} + i A_{i} \right) \\
    \partial_{t} \left( A_{r} + i A_{i} \right) &= \left( B_{r} + i B_{i} \right)
    \end{aligned}
    \tag{24}
\end{equation}

The Real part of the system is: 

\begin{equation}
    \begin{aligned}
        -\frac{\beta_{2}}{2} \partial_{t} B_{i} + v_{g} \partial_{z} A_{r} + \frac{v_{g}}{2 k_{0}} \nabla^{2} A_{i} &= -B_{r} - \frac{\alpha}{2} A_{r} - k_{0} n_{2} I A_{i} \\
        \partial_{t} A_{r} = B_{r}
    \end{aligned}
    \tag{25}
\end{equation}

And the imaginary part:

\begin{equation}
    \begin{aligned}
        \frac{\beta_{2}}{2} \partial_{t} B_{r} + v_{g}\partial_{z} A_{i} - \frac{v_{g}}{2 k_{0}} \nabla^{2} A_{r} &= - B_{i} - \frac{\alpha}{2} A_{i} +  k_{0} n_{2} I A_{r} \\
        \partial_{t} A_{i} &= B_{i}
    \end{aligned}
    \tag{26}
\end{equation}

We re-write to leave the time dependence without multiplication:


\begin{equation}
    \begin{aligned}
         \partial_{t} B_{i} - \frac{2 v_{g}}{\beta_{2}} \partial_{z} A_{r} - \frac{ v_{g}}{ \beta_{2} k_{0}} \nabla^{2} A_{i} &=  \frac{2}{\beta_{2}} B_{r} - \frac{\alpha}{\beta_{2}} A_{r} - \frac{ 2 k_{0} n_{2} }{\beta_{2}} I A_{i} \\
         &\\
        \partial_{t} A_{r} = B_{r}
    \end{aligned}
    \tag{27}
\end{equation}

And the imaginary part:

\begin{equation}
    \begin{aligned}
         \partial_{t} B_{r} + \frac{2 v_{g}}{\beta_{2}}\partial_{z} A_{i} - \frac{v_{g}}{\beta_{2} k_{0}} \nabla^{2} A_{r} &= - \frac{2}{\beta_{2}} B_{i} - \frac{\alpha}{\beta_{2}} A_{i} + \frac{2 k_{0} n_{2}}{\beta_{2}} I A_{r} \\
         &\\
        \partial_{t} A_{i} &= B_{i}
    \end{aligned}
    \tag{28}
\end{equation}

# Creating new Equations in Trixi

**NOTE**: `Trixi.jl` uses $x$, $y$, $z$ for the 1st, 2nd, and 3rd dimension. Most optics books use $z$ as the longitudinal direction and $y$,$z$ for the transverse ($\perp$). Because we're doing a 2D simulation in `Trixi.jl` we will use $x$ and $z$ interchageably for the longitudinal direction, and use $y$, $\perp$, $r$ interchangeably for the transverse direction

## Advection equations

The hyperbolic advection terms of the system of equations are: 

\begin{equation}
    \begin{aligned}
         \partial_{t} B_{i} - \frac{2 v_{g}}{\beta_{2}} \partial_{z} A_{r} &= 0 \\
         &\\
        \partial_{t} A_{r} = 0
    \end{aligned}
    \tag{29}
\end{equation}

And the imaginary part:

\begin{equation}
    \begin{aligned}
         \partial_{t} B_{r} + \frac{2 v_{g}}{\beta_{2}}\partial_{z} A_{i} &= 0 \\
         &\\
        \partial_{t} A_{i} &= 0
    \end{aligned}
    \tag{30}
\end{equation}

We note here that the equations for $B_{r}$ and $B_{i}$ have an advection velocity in the transverse-direction of zero: 

$$v_{B,\perp} \partial_{\perp} B \rightarrow 0$$

For the $A_{r}$ and $A_{i}$ terms, the advection velocity in both directions is zero: 

\begin{equation}
    v_{A,z} \partial_{z} A + v_{A,\perp} \partial_{\perp} A = 0
\end{equation}

Let's write our advection equation into `Trixi.jl`

In [4]:
# Define new physics
using Trixi
using Trixi: AbstractEquations, get_node_vars
import Trixi: varnames, default_analysis_integrals, flux, max_abs_speed_naive,
              have_nonconservative_terms

In [5]:
# Since there is no native support for the Linear Advection equation with two variables
# variables: Ar, Ai, Br, Bi
struct LinearAdvectionEquation4v2D{T} <: AbstractEquations{2 #= spatial dimension =#,
                                                                   4 #= two variables (u,a) =#}
    advection_speed::T #the advection speed for all 4 variables in 2D so it will be an array with 8 total entries
    diffusion_speed::T #the diffusion speed for all 4 variables in 2D so it will be an array with 8 total entries
    k0::T #constant central wavenumber ,for all 4 variables in 2D so it will be an array with 8 total entries
    k0src::T #constant central wavenumber for the source terms ,for all 4 variables in 2D so it will be an array with 4 total entries
    b2::T #\beta_2 group velocity dispersion ,for all 4 variables in 2D so it will be an array with 8 total entries
    b2src::T #\beta_2 group velocity dispersion for the source terms ,for all 4 variables in 2D so it will be an array with 4 total entries
    n2::T #instantaneous nonlinear second order refractive index ,for all 4 variables in 2D so it will be an array with 4 total entries
    alpha::T # optical absorption coefficient,for all 4 variables in 2D so it will be an array with 4 total entries
end

We define our new 2-variable, 2-dimension linear sacalar advection equations as shown above. We create a new struct that is a `Trixi.jl` type of `AbstractEquations` (hence the `<:`). `AbstractEquations{a,b}` is a type where the the `a` is the spatial dimensions of the system, and `b` are the variables of the system. 

Notice that we add the `{T <: Real}` to initialize the struct `LinearAdvectionEquation2v2D` to take in `T` values that belong to the `Real` type of Julia. 

Inside the struct, we want it to hold the speed of light and the central wavenumber: `speed_of_light` and `k0`, respectively. These two parameters will be 2-valued vectors where the first value will be the speed in the longitudinal direction and the second will be the speed in the transverse direction

Next we need to define the variable names of our 2-variable, 2-dimensional linear scalar advection equation. The variables will be the real and imaginary part ($A_{r}$, $A_{i}$). These two variables are the conservative and the primitive so we use multiple-dispatch to overload the `Trixi.jl` function `varnames()`

In [6]:
varnames(::typeof(cons2cons), ::LinearAdvectionEquation4v2D) = ("Ar", "Ai", "Br", "Bi")
varnames(::typeof(cons2prim), ::LinearAdvectionEquation4v2D) = ("Ar", "Ai", "Br", "Bi")

default_analysis_integrals(::LinearAdvectionEquation4v2D) = ()

default_analysis_integrals (generic function with 6 methods)

### Defining Flux

We need to identify the flux for the 2v2D Linear Advection Equations:

\begin{equation}
    \mathbf{F}_{r} = \begin{pmatrix}
    f_{r,z} \\
    f_{r, \perp}
    \end{pmatrix} =  \begin{pmatrix}
    c_{z} \\
    c_{\perp}
    \end{pmatrix} A_{r} 
    \tag{7}
\end{equation}


As shown above, the flux is just simpy the group velocity ($v_{g}$) which in vacuum is just the speed of light. However, because `TreeMesh()` only allows for square domains so we need to normalize the domain in each direction. This is why we need to have 2-values for the speed and wavenumber

In [7]:
# Calculate 2D flux for a single point
@inline function flux(u, orientation::Integer,
                      equations::LinearAdvectionEquation4v2D)
    Ar, Ai, Br, Bi = u

    if orientation == 1
        # advective flux components in the x-direction
        f1 = equations.advection_speed[1] * Ar
        f2 = equations.advection_speed[2] * Ai
        f3 = equations.advection_speed[3] * Ai
        f4 = equations.advection_speed[4] * Ar

        return SVector(f1, f2, f3, f4)
    else # if orientation == 2 the transverse direction
        # advective flux components in the y-direction transverse
        #  in the transverse direction there is no advection 
        g1 = equations.advection_speed[5] * Ar
        g2 = equations.advection_speed[6] * Ai
        g3 = equations.advection_speed[7] * Ai
        g4 = equations.advection_speed[8] * Ar

        return SVector(g1, g2, g3, g4)
    end
end

flux (generic function with 51 methods)

The flux function requires the input variable (which can be a scalar or vector), orientation (z or y directions), and equation type. 

We first unpack the scalar terms from the input variable `u` which should be a vector of the two scalar variables $A_{r}$ and $A_{i}$. 

Next we return a static vector `SVector()` of the fluxes as shown above.

### Defining Speed Limits

We need to calculate the maximum speed possible for our fluxes. We do this by multiple-dispatching the `Trixi.jl` function `max_abs_speed_naive()`. 

In [8]:
# Calculate maximum wave speed for local Lax-Friedrichs-type dissipation
@inline function max_abs_speed_naive(u_ll, u_rr, orientation::Int,
                                     equations::LinearAdvectionEquation4v2D)
    cd = 0.0
    if orientation == 1
        cd = equations.advection_speed[1]
    end
    λ_max = abs(cd)
end

@inline have_constant_speed(::LinearAdvectionEquation4v2D) = True()

@inline function max_abs_speeds(equations::LinearAdvectionEquation4v2D)
    if equations.orientation == 1
        return SVector(equations.advection_speed[1], equations.advection_speed[2], equations.advection_speed[3], equations.advection_speed[4])
    else
        return SVector(equations.advection_speed[5], equations.advection_speed[6], equations.advection_speed[7], equations.advection_speed[8])
    end
end

max_abs_speeds (generic function with 1 method)

Additionally, we need to set the maximum absolute speed possible using the `max_abs_speeds()`. Again, this is a `Trixi.jl` function so it will be multiple-dispatched with the `equations::LinearAdvectionEquation2v2D` input type. In this function, it must return the maximum absolute speed for the two variables ($A_r$, $A_i$). THe return function should be a 2-valued `SVector` corresponding to the speeds of $A_r$ and $A_i$. Additionally, the speeds should depend on the direction of the flux (`orientation`).

## Parabolic Diffusion Terms (Diffraction)

The parabolic diffusion (diffraction) terms of tour system is: 

\begin{equation}
    \begin{aligned}
        \partial_{t} A_{r} &= 0 \\
        \partial_{t} A_{i} &= 0 \\
        \partial_{t} B_{r} - \frac{v_{g}}{\beta_{2} k_{0}} \nabla^{2} A_{r} &= 0 \\
        \partial_{t} B_{i} - \frac{v_{g}}{\beta_{2} k_{0}} \nabla^{2} A_{i} &= 0
    \end{aligned}
    \tag{31}
\end{equation}



### Creating Parabolic Equation Struct

We need to make an equation struct to hold all the information for the diffraction term. We can use the code from the tutorial: https://trixi-framework.github.io/Trixi.jl/stable/tutorials/adding_new_parabolic_terms/

In [9]:
struct ConstantAnisotropicDiffusion2D{E, T} <: Trixi.AbstractEquationsParabolic{2, 2, GradientVariablesConservative}
    # diffusivity::T
    equations_hyperbolic::E
    advection_speed::T #the advection speed for all 4 variables in 2D so it will be an array with 8 total entries
    diffusion_speed::T #the diffusion speed for all 4 variables in 2D so it will be an array with 8 total entries
    k0::T #constant central wavenumber ,for all 4 variables in 2D so it will be an array with 8 total entries
    k0src::T #constant central wavenumber for the source terms ,for all 4 variables in 2D so it will be an array with 4 total entries
    b2::T #\beta_2 group velocity dispersion ,for all 4 variables in 2D so it will be an array with 8 total entries
    b2src::T #\beta_2 group velocity dispersion for the source terms ,for all 4 variables in 2D so it will be an array with 4 total entries
    n2::T #instantaneous nonlinear second order refractive index ,for all 4 variables in 2D so it will be an array with 4 total entries
    alpha::T # optical absorption coefficient,for all 4 variables in 2D so it will be an array with 4 total entries
end

Creating the abstract struct: 

`struct ConstantAnisotropicDiffusion2D{E, T}`

where the `E` is the abstract type which will be used to refer to the `equation` type, and the `T` will be used to refer to the diffusivity type. In this case, because it is 2D it will be a 2D vector type.

This abstract struct allows for anisotropic diffusion in the longitudinal and transverse direction. In this case, the SVEA is not anisotropic but we will leave it in case users want to play around with it. 

This struct will belong to (`<:`):

`Trixi.AbstractEquationsParabolic{2, 1, GradientVariablesConservative}`

Where the `2` is the number of dimensions, `1` is the number of variables, and `GradientVariablesConservative` is chosen because the hyperbolic system is conservative. This is the case because there is no dissipation mechanism (i.e. optical absorption from air).

Next we need to map the parabolic variables to the hyperbolic variables. `Trixi.jl` already has a function to obtain the variable names: `varnames()`. Since the `LinearScalarAdvectionEquation2D` is already part of the `Trixi.jl` package and part of the `equation` abstract type, `varnames()` will be able to act and return something for the hyperbolic terms. 

We need to define a function via multiple-dispatch to do the same for our parabolic terms. We do this below:

In [10]:
varnames(variable_mapping, equations_parabolic::ConstantAnisotropicDiffusion2D) =
  varnames(variable_mapping, equations_parabolic.equations_hyperbolic)

varnames (generic function with 73 methods)

In this new multiple-dispatched `varnames()`, the first input argument is the same as the the one from the `Trixi.jl` package. `variable_mapping` is the map from one set of variables to the other. 

The second argument then defines our multiple-dispatch where we have define this function to take effect when the second argument is a type `::ConstantAnisotropicDiffusion2D`. When this is the case, it actually just calls the original `varnames()` and inputs the `ConstantAnisotropicDiffusion2D` struct's hyperbolic equations which we defined in the abstract struct above!

### Parabolic Flux

Next, we define the viscous flux function. We assume that the mixed hyperbolic-parabolic system is of the form:

\begin{equation}
    \frac{\partial}{\partial t} A + \frac{\partial}{\partial x} \left[ f_{1}\left( A \right) + g_{1}\left( A, \nabla A \right)  \right] + \frac{\partial}{\partial y} \left[ f_{2}\left( A \right) + g_{2}\left( A, \nabla A \right)  \right] = 0
    \tag{10}
\end{equation}

where $f_{1}(A)$ and $f_{2}(A)$ are the hyperbolic fluxes. $g_{1}(A, \nabla A)$ and $g_{2}(A \nabla A)$ denotes the viscous fluxes. For anisotropic diffusion, the viscous fluxes are the first and second components of the matrix-vector product involving `diffusivity` and the gradient vector. Here, we specialize the flux to our new parabolic equation type `ConstantAnisotropicDiffusion2D`

However, because we have to scalar values ($A_{r} $, $A_{i}$), we'll follow along the `compressible_navier_stokes_2d.jl` source code, specifically the parabolic diffusion flux for the `CompressibleNavierStokesDiffusion2D`.

https://github.com/trixi-framework/Trixi.jl/blob/1d7b2561ff656cb9b8c826ee8b7ee389a3370917/src/equations/compressible_navier_stokes_2d.jl

In our case, the viscous stress tensor is anisotropic such that: 

\begin{equation}
    \tau_{z} = -\frac{v_{g,z}}{\beta_{2,z} k_{0,z}}\begin{bmatrix}
    0 & 0 & 0 & 0\\
    0 & 0  & 0 & 0 \\
    1 & 0  & 0 & 0 \\
    0 & 1  & 0 & 0 \\
    \end{bmatrix}
    \tag{32}
\end{equation}

and

\begin{equation}
    \tau_{r} = -\frac{v_{g,r}}{\beta_{2,r} k_{0,r}}\begin{bmatrix}
    0 & 0 & 0 & 0\\
    0 & 0  & 0 & 0 \\
    1 & 0  & 0 & 0 \\
    0 & 1  & 0 & 0 \\
    \end{bmatrix}
    \tag{33}
\end{equation}

so that the diffusive flux in the z-direction is: 

\begin{equation}
    \tau_{z}\cdot \begin{pmatrix}
    A_{r} \\
    A_{i} \\
    B_{r} \\
    B_{i}
    \end{pmatrix} = -\frac{v_{g,z}}{\beta_{2,z} k_{0,z}} \begin{pmatrix}
    0 \\
    0 \\
    A_{r} \\
    A_{i}
    \end{pmatrix}
    \tag{34}
\end{equation}

and the diffusive flux in the transverse y-direction is: 

\begin{equation}
    \tau_{r}\cdot \begin{pmatrix}
    A_{r} \\
    A_{i} \\
    B_{r} \\
    B_{i}
    \end{pmatrix} = -\frac{v_{g,r}}{\beta_{2,r} k_{0,r}} \begin{pmatrix}
    0 \\
    0 \\
    A_{r} \\
    A_{i}
    \end{pmatrix}
    \tag{35}
\end{equation}

In [11]:
# Explicit formulas for the diffusive Navier-Stokes fluxes are available, e.g., in Section 2
# of the paper by Rueda-Ramírez, Hennemann, Hindenlang, Winters, and Gassner
# "An Entropy Stable Nodal Discontinuous Galerkin Method for the resistive
#  MHD Equations. Part II: Subcell Finite Volume Shock Capturing"
# where one sets the magnetic field components equal to 0.
function flux(u, gradients, orientation::Integer, equations::ConstantAnisotropicDiffusion2D)
    
    # Here `gradients` is assumed to contain the gradients of the primitive variables (rho, v1, v2, T)
    # either computed directly or reverse engineered from the gradient of the entropy variables
    # by way of the `convert_gradient_variables` function.
    dArdz, dAidz, dBrdz, dBidz = gradients[1]
    dArdy, dAidy, dBrdy, dBidy = gradients[2]

    # Components of viscous stress tensor

    tauz_11 = 0.0
    tauz_12 = 0.0
    tauz_13 = 0.0
    tauz_14 = 0.0
    
    tauz_21 = 0.0
    tauz_22 = 0.0
    tauz_23 = 0.0
    tauz_24 = 0.0

    tauz_31 = 1.0
    tauz_32 = 0.0
    tauz_33 = 0.0
    tauz_34 = 0.0

    tauz_41 = 0.0
    tauz_42 = 1.0
    tauz_43 = 0.0
    tauz_44 = 0.0
    
    taur_11 = 0.0
    taur_12 = 0.0
    taur_13 = 0.0
    taur_14 = 0.0
    
    taur_21 = 0.0
    taur_22 = 0.0
    taur_23 = 0.0
    taur_24 = 0.0

    taur_31 = 1.0
    taur_32 = 0.0
    taur_33 = 0.0
    taur_34 = 0.0

    taur_41 = 0.0
    taur_42 = 1.0
    taur_43 = 0.0
    taur_44 = 0.0
    
    if orientation == 1
        # viscous flux components in the x-direction
        f1 = 0.0
        f2 = 0.0
        f3 = tauz_31 * dArdz
        f4 = tauz_42 * dAidz

        return SVector(f1, f2, f3, f4)
    else # if orientation == 2
        # viscous flux components in the y-direction
        g1 = 0.0
        g2 = 0.0
        g3 = taur_31 * dArdy
        g4 = taur_42 * dAidy

        return SVector(g1, g2, g3, g4)
    end
end

flux (generic function with 52 methods)

# Source Terms

The source terms are the optical absorption, nonlinear Kerr terms along with the time derivatives of our amplitudes:

\begin{equation}
    \begin{aligned}
        \partial_{t} A_{r} &= B_{r} \\
        \partial_{t} A_{i} &= B_{i} \\
        \partial_{t} B_{r} &= -\frac{2}{\beta_{2}} B_{i} - \frac{\alpha}{\beta_{2}} A_{i} + \frac{2 k_{0} n_{2}}{\beta_{2}} I A_{r} \\
        \partial_{t} B_{i} &= -\frac{2}{\beta_{2}} B_{r} - \frac{\alpha}{\beta_{2}} A_{r} + \frac{2 k_{0} n_{2}}{\beta_{2}} I A_{i}
    \end{aligned}
    \tag{35}
\end{equation}


THe documentation of `Trixi.jl` has a page on adding source terms such as those above: https://trixi-framework.github.io/Trixi.jl/stable/tutorials/custom_semidiscretization/#Using-a-custom-ODE-right-hand-side-function

We need to define a function with the name `source_terms_standard(u,x,t,equations)` and it should return an `SVector`:

In [12]:
function source_terms_standard(u, x, t, equations::LinearAdvectionEquation4v2D)

    Ar, Ai, Br, Bi = u

    #intensity
    Aint = Ar^2 + Ai^2

    Arsrc = Br #source term for Ar
    Aisrc = Bi #source term for Ai
    
    Brsrc = -0.5*equations.b2src[3]*Bi - 0.5*equations.alpha[3]/equations.b2src[3]*Ai + 2.0*equations.k0src[3]*equations.n2[3]/equations.b2src[3]*Aint*Ar

    Bisrc = -0.5*equations.b2src[4]*Br - 0.5*equations.alpha[4]/equations.b2src[4]*Ar + 2.0*equations.k0src[4]*equations.n2[4]/equations.b2src[4]*Aint*Ai
    
    
    return SVector(Arsrc, Aisrc, Brsrc, Bisrc)
end

source_terms_standard (generic function with 1 method)

# Putting it All Together

## Initialize Equations

Let's initialize the hyperbolic advection equations. We need to define the speed of light and wavenumber. Let's also define the central wavelength and central frequency of our laser.

We define the group velocity as the group velocity in vacuum: $v_g = c$

While $k_0$ is the central wave number. We'll use a central wavelength of $\lambda_0 = 1 \mu \mathrm{m}$

In [13]:
c = 2.998e8 #m/s speed of light in vacuum
l0 = 1.0e-6 #central wavelength in meters
k0 = 2.0*pi/l0 #central wavenumber
omega0 = c*k0 ; #central angular frequency (2*pi f)
beta2 = 1.0 # beta_2 group velocity dispersion function
a = 1.0 # alpha optical absorption coefficient
N2 = 1.0 # instantaneous second order nonliner index of refraction

We'll use a material with relatively high absorption and dispersion in order to see some actual results. 

We'll use the wavelength as the normalization for the space variables (z,y)

Let's now define our advection equation. We will normalize the speed to the speed of light and the length to our total length of the simulation. Let's say we want to see the laser travel: 

$$ Z_{norm} = 10 \:\mathrm{mm}$$

and we want the radial normalization:

$$ R_{norm} = 2 \:\mathrm{mm}$$

This will be our normalization lengths. THis comes into play in the **Anisotropic Flux** in both the hyperbolic and parabolic equations . So then our light speed and wavenumber needs to be normalized in the longitudinal and transverse direction as follows: 

In [14]:
Znorm = 10.0e-2 #normalization factor in longitudinal direction
Rnorm = 0.002 #normalizaiton in transverse
l0znorm = l0/Znorm #normalized wavelength
l0rnorm = l0/Rnorm #normalized wavelength
k0znorm = 2.0*pi/l0znorm #normalized wavenumber
k0rnorm = 2.0*pi/l0rnorm #normalized wavenumber

cznorm = 1.0 #normalized speed of light
Tnorm = Znorm/c
crnorm = c*Tnorm/l0rnorm

omega0norm = omega0*Tnorm

# println("Normalization Length: ", Lnorm)
# println("Normalized wavelength: ", l0norm)
# println("Normalized wavenumber: ", k0norm)
# println("Normalized speed of light: ", cnorm)
# println("Normalization time: ", Tnorm)

628318.5307179588

In [ ]:
advspeed = (0.0, 0.0, 0.0, 0.0, -2.0/beta2 * cznorm, -2.0/beta2 * crnorm, 2.0/beta2 * cznorm, -2.0/beta2 * crnorm)

difspeed = (0.0, 0.0, 0.0, 0.0, -cznorm/beta2/k0znorm, -crnorm/beta2/k0rnorm, -cznorm/beta2/k0znorm, -crnorm/beta2/k0rnorm)

k0norm = (k0znorm, k0rnorm, k0znorm, k0rnorm, k0znorm, k0rnorm, k0znorm, k0rnorm)

k0src = (k0znorm, k0rnorm, k0znorm, k0rnorm)

b2 = (beta2, beta2, beta2, beta2, beta2, beta2, beta2, beta2)

b2src = (beta2, beta2, beta2, beta2)

n2 = (N2, N2, N2, N2)
alpha = (0.0, 0.0, a, a, a, a)

Notice that we also normalized time by the speed of light and total distance of travel. We will use this for the initial conditions

Now let's initialize our hyperbolic equations

In [ ]:
equations_hyperbolic = LinearAdvectionEquation4v2D()

Note that in defining `equations_hyperbolic` we use to inputs into our custom `LinearAdvectionEquation2v2D()` function. This is because the parametric struct has two variables within it as we defined it: `speed_of_light` and `k0`. This means that we can initialize these parameters of the struct by inputting them as arguments in chronological order. 

Let's now initialize the parabolic part of our equation. Similar to the hyperbolic initialization, we input the arguments for the hyperbolic equations, the speed of light, and wavenumber

In [ ]:
equations_parabolic = ConstantAnisotropicDiffusion2D(equations_hyperbolic, cnorm, k0norm)